In [2]:
from joblib import dump, load
import pandas as pd
import numpy as np
import os
import seaborn as sns
from mlni.base import RB_Input
from matplotlib.pyplot import show
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from copy import deepcopy
from sklearn.svm import SVR
from sklearn.metrics.pairwise import rbf_kernel
import json

# LASSO

In [7]:
### NOTE: need to use the same python version to load the joblib
### USE mlni conda virtual env

def apply_lasso(model_joblib_dir, data_dir, organ):
    """
    This is to apply the trained lasso model to the test data of UKBB and derive the BA
    :param saved_model:
    :param output_dir:
    :param train_tsv:
    :return:
    """

    ## test
    print('MLNI for applying model to test for organ BAG: %s...' % organ)
    test_tsv = os.path.join(data_dir, f"followup_data_{organ}.tsv")
    
    df_test = pd.read_csv(test_tsv, sep='\t')[['participant_id', 'session_id', 'diagnosis']]
    input_data = RB_Input(test_tsv, standardization_method="minmax")

    feature = input_data.get_x()

    # load the model
    model_joblib = os.path.join(model_joblib_dir, organ, 'regression', 'regressor', 'lasso_single.joblib')
    output_dir = os.path.join(model_joblib_dir, organ, 'regression')
    model = load(model_joblib)

    y_hat = model.predict(feature)
    df_test['predicted_age'] = y_hat

    ### plot the scatterplot
    # sns.scatterplot(df_test['diagnosis'], df_test["predicted_age"])
    # ax = sns.regplot(x="diagnosis", y="predicted_age", data=df_test, scatter=False,
    # fit_reg=True, color='red')
    # ax.set_xlim(30, 100)
    # ax.set_ylim(30, 100)
    # ax.plot([0, 1], [0, 1], '--', transform=ax.transAxes, color="black")
    # show()
    ### calculate the mae and pearson correlation
    mae = mean_absolute_error(df_test['diagnosis'], df_test["predicted_age"])
    r, _ = pearsonr(df_test['diagnosis'], df_test["predicted_age"])
    pd.DataFrame.from_dict({'MAE': [mae], 'r': [r]}).to_csv(os.path.join(output_dir, 'metrics_test_lasso.tsv'), index=False, sep='\t')

    df_test.to_csv(os.path.join(output_dir, 'results_test_lasso.tsv'), index=False, sep='\t', encoding='utf-8')

    ### correct the brain age prediction using Beheshti et al.  method: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7049655/
    df_test_corrected = deepcopy(df_test)
    df_test_corrected['bag'] = df_test_corrected['predicted_age'] - df_test_corrected['diagnosis']
    reg = LinearRegression().fit(df_test_corrected['diagnosis'].to_numpy()[..., np.newaxis], df_test_corrected['bag'].to_numpy())
    alpha = reg.coef_[0]
    beta = reg.intercept_
    pd.DataFrame.from_dict({'alpha': [alpha], 'beta': [beta]}).to_csv(os.path.join(output_dir, 'age_correction_param_from_test.tsv'), index=False, sep='\t')
    df_test_corrected['predicted_age'] = df_test_corrected['predicted_age'] - (alpha * df_test_corrected['diagnosis'] + beta)
    del df_test_corrected['bag']

    ### plot the scatterplot after correction
    # sns.scatterplot(df_test_corrected['diagnosis'], df_test_corrected["predicted_age"])
    # ax = sns.regplot(x="diagnosis", y="predicted_age", data=df_test_corrected, scatter=False,
    # fit_reg=True, color='red')
    # ax.set_xlim(30, 100)
    # ax.set_ylim(30, 100)
    # ax.plot([0, 1], [0, 1], '--', transform=ax.transAxes, color="black")
    # show()

    ### calculate the mae and pearson correlation
    mae_corrected = mean_absolute_error(df_test_corrected['diagnosis'], df_test_corrected["predicted_age"])
    r_corrected, _ = pearsonr(df_test_corrected['diagnosis'], df_test_corrected["predicted_age"])
    pd.DataFrame.from_dict({'MAE': [mae_corrected], 'r': [r_corrected]}).to_csv(os.path.join(output_dir, 'metrics_test_corrected_lasso.tsv'), index=False, sep='\t')

    df_test_corrected.to_csv(os.path.join(output_dir, 'results_test_corrected_lasso.tsv'), index=False,
                              sep='\t', encoding='utf-8')

    print("Stop here...")

In [8]:
for organ in ['eye', 'heart', 'endocrine', 'pulmonary', 'immune', 'hepatic', 'repro_male', 'repro_female', 'skin', 'renal', 'panceas', 'brain', 'digestive', 'salivary']:
    model_joblib_dir = "output/Lasso"
    data_dir = 'data'
    apply_lasso(model_joblib_dir, data_dir, organ)

MLNI for applying model to test for organ BAG: eye...
Stop here...
MLNI for applying model to test for organ BAG: heart...
Stop here...
MLNI for applying model to test for organ BAG: endocrine...
Stop here...
MLNI for applying model to test for organ BAG: pulmonary...
Stop here...
MLNI for applying model to test for organ BAG: immune...
Stop here...
MLNI for applying model to test for organ BAG: hepatic...
Stop here...
MLNI for applying model to test for organ BAG: repro_male...
Stop here...
MLNI for applying model to test for organ BAG: repro_female...
Stop here...
MLNI for applying model to test for organ BAG: skin...
Stop here...
MLNI for applying model to test for organ BAG: renal...
Stop here...
MLNI for applying model to test for organ BAG: panceas...
Stop here...
MLNI for applying model to test for organ BAG: brain...
Stop here...
MLNI for applying model to test for organ BAG: digestive...
Stop here...
MLNI for applying model to test for organ BAG: salivary...
Stop here...


# Elastic Net

In [9]:
### NOTE: need to use the same python version to load the joblib
### USE mlni conda virtual env

def apply_en(model_joblib_dir, data_dir, organ):
    """
    This is to apply the trained lasso model to the test data of UKBB and derive the BA
    :param saved_model:
    :param output_dir:
    :param train_tsv:
    :return:
    """

    ## test
    print('MLNI for applying model to test for organ BAG: %s...' % organ)
    test_tsv = os.path.join(data_dir, f"followup_data_{organ}.tsv")
    
    df_test = pd.read_csv(test_tsv, sep='\t')[['participant_id', 'session_id', 'diagnosis']]
    input_data = RB_Input(test_tsv, standardization_method="minmax")

    feature = input_data.get_x()

    # load the model
    model_joblib = os.path.join(model_joblib_dir, organ, 'regression', 'regressor', 'elasticnet_single.joblib')
    output_dir = os.path.join(model_joblib_dir, organ, 'regression')
    model = load(model_joblib)

    y_hat = model.predict(feature)
    df_test['predicted_age'] = y_hat

    ### plot the scatterplot
    # sns.scatterplot(df_test['diagnosis'], df_test["predicted_age"])
    # ax = sns.regplot(x="diagnosis", y="predicted_age", data=df_test, scatter=False,
    # fit_reg=True, color='red')
    # ax.set_xlim(30, 100)
    # ax.set_ylim(30, 100)
    # ax.plot([0, 1], [0, 1], '--', transform=ax.transAxes, color="black")
    # show()
    ### calculate the mae and pearson correlation
    mae = mean_absolute_error(df_test['diagnosis'], df_test["predicted_age"])
    r, _ = pearsonr(df_test['diagnosis'], df_test["predicted_age"])
    pd.DataFrame.from_dict({'MAE': [mae], 'r': [r]}).to_csv(os.path.join(output_dir, 'metrics_test_lasso.tsv'), index=False, sep='\t')

    df_test.to_csv(os.path.join(output_dir, 'results_test_lasso.tsv'), index=False, sep='\t', encoding='utf-8')

    ### correct the brain age prediction using Beheshti et al.  method: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7049655/
    df_test_corrected = deepcopy(df_test)
    df_test_corrected['bag'] = df_test_corrected['predicted_age'] - df_test_corrected['diagnosis']
    reg = LinearRegression().fit(df_test_corrected['diagnosis'].to_numpy()[..., np.newaxis], df_test_corrected['bag'].to_numpy())
    alpha = reg.coef_[0]
    beta = reg.intercept_
    pd.DataFrame.from_dict({'alpha': [alpha], 'beta': [beta]}).to_csv(os.path.join(output_dir, 'age_correction_param_from_test.tsv'), index=False, sep='\t')
    df_test_corrected['predicted_age'] = df_test_corrected['predicted_age'] - (alpha * df_test_corrected['diagnosis'] + beta)
    del df_test_corrected['bag']

    ### plot the scatterplot after correction
    # sns.scatterplot(df_test_corrected['diagnosis'], df_test_corrected["predicted_age"])
    # ax = sns.regplot(x="diagnosis", y="predicted_age", data=df_test_corrected, scatter=False,
    # fit_reg=True, color='red')
    # ax.set_xlim(30, 100)
    # ax.set_ylim(30, 100)
    # ax.plot([0, 1], [0, 1], '--', transform=ax.transAxes, color="black")
    # show()

    ### calculate the mae and pearson correlation
    mae_corrected = mean_absolute_error(df_test_corrected['diagnosis'], df_test_corrected["predicted_age"])
    r_corrected, _ = pearsonr(df_test_corrected['diagnosis'], df_test_corrected["predicted_age"])
    pd.DataFrame.from_dict({'MAE': [mae_corrected], 'r': [r_corrected]}).to_csv(os.path.join(output_dir, 'metrics_test_corrected_lasso.tsv'), index=False, sep='\t')

    df_test_corrected.to_csv(os.path.join(output_dir, 'results_test_corrected_lasso.tsv'), index=False,
                              sep='\t', encoding='utf-8')

    print("Stop here...")

In [10]:
for organ in ['eye', 'heart', 'endocrine', 'pulmonary', 'immune', 'hepatic', 'repro_male', 'repro_female', 'skin', 'renal', 'panceas', 'brain', 'digestive', 'salivary']:
    model_joblib_dir = "output/EN"
    data_dir = 'data'
    apply_en(model_joblib_dir, data_dir, organ)

MLNI for applying model to test for organ BAG: eye...
Stop here...
MLNI for applying model to test for organ BAG: heart...
Stop here...
MLNI for applying model to test for organ BAG: endocrine...
Stop here...
MLNI for applying model to test for organ BAG: pulmonary...
Stop here...
MLNI for applying model to test for organ BAG: immune...
Stop here...
MLNI for applying model to test for organ BAG: hepatic...
Stop here...
MLNI for applying model to test for organ BAG: repro_male...
Stop here...
MLNI for applying model to test for organ BAG: repro_female...
Stop here...
MLNI for applying model to test for organ BAG: skin...
Stop here...
MLNI for applying model to test for organ BAG: renal...
Stop here...
MLNI for applying model to test for organ BAG: panceas...
Stop here...
MLNI for applying model to test for organ BAG: brain...
Stop here...
MLNI for applying model to test for organ BAG: digestive...
Stop here...
MLNI for applying model to test for organ BAG: salivary...
Stop here...


# SVM

In [4]:
def launch_svr(x_train, x_test, y_train, c,gamma):
    svr = SVR(C=c, kernel='precomputed', #max_iter=1e6, 
                  tol=1e-2)
    kernel_train = rbf_kernel(x_train, x_train, gamma=gamma)
    svr.fit(kernel_train, y_train)
    kernel_test = rbf_kernel(x_test, x_train, gamma=gamma)
    y_hat = svr.predict(kernel_test)
    return y_hat
def find_key(obj, key):
    if isinstance(obj, dict):
        if key in obj:
            return obj[key]
        for v in obj.values():
            res = find_key(v, key)
            if res is not None:
                return res
    elif isinstance(obj, list):
        for item in obj:
            res = find_key(item, key)
            if res is not None:
                return res
    return None

In [ ]:
### NOTE: need to use the same python version to load the joblib
### USE mlni conda virtual env

def apply_svm(model_joblib_dir, data_dir, organ):
    """
    This is to apply the trained lasso model to the test data of UKBB and derive the BA
    :param saved_model:
    :param output_dir:
    :param train_tsv:
    :return:
    """

    ## test
    print('MLNI for applying model to test for organ BAG: %s...' % organ)
    train_tsv = os.path.join(data_dir, f"training_data_{organ}.tsv")
    test_tsv = os.path.join(data_dir, f"followup_data_{organ}.tsv")
    
    df_test = pd.read_csv(test_tsv, sep='\t')[['participant_id', 'session_id', 'diagnosis']]
    train_input = RB_Input(train_tsv, standardization_method="minmax")
    test_input = RB_Input(test_tsv, standardization_method="minmax")

    train_feature = train_input.get_x()
    test_feature = test_input.get_x()

    # load the model
    para_path = os.path.join(model_joblib_dir, organ, 'regression', 'regressor','best_parameters.json')
    with open(para_path, "r") as f:
        para_data = json.load(f)

    best_gamma_single = find_key(para_data, "gamma_single")
    best_c_single = find_key(para_data, "c_single")

    output_dir = os.path.join(model_joblib_dir, organ, 'regression')

    y_hat = launch_svr(train_feature, 
                       test_feature, 
                       train_input.get_y_raw(), 
                       c = best_c_single,
                       gamma = best_gamma_single)
    y_hat = list(y_hat)
    df_test['predicted_age'] = y_hat

    ### plot the scatterplot
    # sns.scatterplot(df_test['diagnosis'], df_test["predicted_age"])
    # ax = sns.regplot(x="diagnosis", y="predicted_age", data=df_test, scatter=False,
    # fit_reg=True, color='red')
    # ax.set_xlim(30, 100)
    # ax.set_ylim(30, 100)
    # ax.plot([0, 1], [0, 1], '--', transform=ax.transAxes, color="black")
    # show()
    ### calculate the mae and pearson correlation
    mae = mean_absolute_error(df_test['diagnosis'], df_test["predicted_age"])
    r, _ = pearsonr(df_test['diagnosis'], df_test["predicted_age"])
    pd.DataFrame.from_dict({'MAE': [mae], 'r': [r]}).to_csv(os.path.join(output_dir, 'metrics_test_lasso.tsv'), index=False, sep='\t')

    df_test.to_csv(os.path.join(output_dir, 'results_test_lasso.tsv'), index=False, sep='\t', encoding='utf-8')

    ### correct the brain age prediction using Beheshti et al.  method: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7049655/
    df_test_corrected = deepcopy(df_test)
    df_test_corrected['bag'] = df_test_corrected['predicted_age'] - df_test_corrected['diagnosis']
    reg = LinearRegression().fit(df_test_corrected['diagnosis'].to_numpy()[..., np.newaxis], df_test_corrected['bag'].to_numpy())
    alpha = reg.coef_[0]
    beta = reg.intercept_
    pd.DataFrame.from_dict({'alpha': [alpha], 'beta': [beta]}).to_csv(os.path.join(output_dir, 'age_correction_param_from_test.tsv'), index=False, sep='\t')
    df_test_corrected['predicted_age'] = df_test_corrected['predicted_age'] - (alpha * df_test_corrected['diagnosis'] + beta)
    del df_test_corrected['bag']

    ### plot the scatterplot after correction
    # sns.scatterplot(df_test_corrected['diagnosis'], df_test_corrected["predicted_age"])
    # ax = sns.regplot(x="diagnosis", y="predicted_age", data=df_test_corrected, scatter=False,
    # fit_reg=True, color='red')
    # ax.set_xlim(30, 100)
    # ax.set_ylim(30, 100)
    # ax.plot([0, 1], [0, 1], '--', transform=ax.transAxes, color="black")
    # show()

    ### calculate the mae and pearson correlation
    mae_corrected = mean_absolute_error(df_test_corrected['diagnosis'], df_test_corrected["predicted_age"])
    r_corrected, _ = pearsonr(df_test_corrected['diagnosis'], df_test_corrected["predicted_age"])
    pd.DataFrame.from_dict({'MAE': [mae_corrected], 'r': [r_corrected]}).to_csv(os.path.join(output_dir, 'metrics_test_corrected_lasso.tsv'), index=False, sep='\t')

    df_test_corrected.to_csv(os.path.join(output_dir, 'results_test_corrected_lasso.tsv'), index=False,
                              sep='\t', encoding='utf-8')

    print("Stop here...")

In [8]:
for organ in ['eye', 'heart', 'endocrine', 'pulmonary', 'immune', 'hepatic', 'repro_male', 'repro_female', 'skin', 'renal', 'panceas', 'brain', 'digestive', 'salivary']:
    model_joblib_dir = "output/SVM"
    data_dir = 'data'
    apply_svm(model_joblib_dir, data_dir, organ)

MLNI for applying model to test for organ BAG: eye...
Stop here...
MLNI for applying model to test for organ BAG: heart...
Stop here...
MLNI for applying model to test for organ BAG: endocrine...
Stop here...
MLNI for applying model to test for organ BAG: pulmonary...
Stop here...
MLNI for applying model to test for organ BAG: immune...
Stop here...
MLNI for applying model to test for organ BAG: hepatic...
Stop here...
MLNI for applying model to test for organ BAG: repro_male...
Stop here...
MLNI for applying model to test for organ BAG: repro_female...
Stop here...
MLNI for applying model to test for organ BAG: skin...
Stop here...
MLNI for applying model to test for organ BAG: renal...
Stop here...
MLNI for applying model to test for organ BAG: panceas...
Stop here...
MLNI for applying model to test for organ BAG: brain...
Stop here...
MLNI for applying model to test for organ BAG: digestive...
Stop here...
MLNI for applying model to test for organ BAG: salivary...
Stop here...
